# 4. Tools and Agents

## Syllabus
* 4.1 Introduction to Tools
* 4.2 Introduction to Agents
* 4.3 Built-in Tools in LangChain
* 4.4 Agents in Action
* 4.5 Building a Multi-Tool Agent
* 4.6 Creating Custom Tools

### 4.1 Introduction to Tools
A tool in LangChain is simply a function that LLM uses when it cannot do on its own.
eg. A calculator is a tool that an LLM can use because LLM's can make math mistakes

Every tool has 3 main parts
* Name: Unique identifier used by the agent
* Description: Natural language instruction so the agent knows when to use it
* Function: Actual python function

## 4.2 Built-in Tools in LangChain

#### Tool Example 1: Calculator Tool

* `PythonREPLTool()` gives the LLM access to a Python REPL(read, eval, print, loop)
* It executes the given string as a python code
* creates a small sandbox python interpretor
* does not require an LLM


In [ ]:
import os
import sys
from pathlib import Path

sys.path.append(os.path.abspath(".."))

In [ ]:
# initializing the llm
from llm.load_llm import initialize_llm

llm = initialize_llm()

`pip install langchain-experimental`

In [ ]:
from langchain_experimental.tools import PythonREPLTool

calculator = PythonREPLTool()

# Add print() to see the results
print(calculator.run("print(5*12+(33/3))"))
print(calculator.run("print(max(25,69))"))
print(calculator.run("print(sum([i for i in range(1,11)]))"))

`LLMMathChain`
* Instead of directly executing code, it asks the LLM to write math as python code, then evaluate it.

In [ ]:
from langchain.chains import LLMMathChain

math_chain = LLMMathChain.from_llm(llm = llm, verbose = True)
print(math_chain.run("What is (5*12+(33/3))"))

#### Tool Example 2: Search Tool
* LLM's don't have access to fresh information like daily news, stock prices, weather etc
* To fix this, we connect them to a search API.

* There are a few tools for searching
    - Tavily Search API
    - SerpAPI
    - DuckDuckGoSearch

`pip install tavily-python`

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
tavily_key = os.environ.get("TAVILY_API_KEY")

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results = 2)

results = search.run("What is the latest weather in Kochi, Kerala, India")
print(results)

In [ ]:
print(results[0]["content"])

* Alternatively we can use `DuckDuckGo search` 

### Using Search in an Agent
📌 Here’s what happens:
- LLM sees the question.
- Decides → “I don’t know Bitcoin’s price, I must use search.”
- Calls search tool → fetches live price.
- Passes that result into the calculator.
- Returns final answer.

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, AgentType

# Give the agent tools (search + calculator)
tools = [search, calculator]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Now the LLM can decide when to search
agent.run("What is the current price of Bitcoin in USD divided by 3?")


## 4.3 Creating Custom Tools

In [ ]:
from langchain.tools import tool

# define a custom tool with a decorator function
@tool
def get_weather(city: str) -> str:
    """ Get the weather for a given city from these dummy data"""
    weather_data = {
        "New York": "Sunny, 25°C",
        "London": "Rainy, 15°C",
        "Tokyo": "Cloudy, 20°C"
    }

    return weather_data[city]

using it in an agent

In [ ]:
tools = [get_weather]

agent2 = initialize_agent(
    tools = tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose= True
)

agent2.run("Whats the weather in London")

In [ ]:
import numpy as np
from langchain.tools import tool

@tool
def matrix_determinant(matrix: str)->str:
    """
    Calculates the determinant of a 2x2 matrix
    Input: 2x2 matrix in string format
    Output: determinant value in string
    """
    arr = np.array(eval(matrix))
    det = np.linalg.det(arr)
    return f"The determinant is {det}"

In [ ]:
from langchain.agents import initialize_agent, AgentType

tools3 = [matrix_determinant]
agent3 = initialize_agent(
    llm = llm,
    tools = tools3,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True
)

agent3.run("Determine the determinant of matrix [[1,2],[3,4]]")

## 4.4 Introduction to Agents

An agent is like an AI decision maker that
* takes in a user query as input
* Decides how to solve it (tool or reasoning)
* uses the chosen tools
* returns the final answer

eg. What is the square root of population of Japan.
* Pick at tool:
    - Search tool to get population
    - Calculator tool to get square root
    - Database tool

#### ReAct Framework (Reason + Act)
* The agent reasons step by step
* Then it acts by using tools.

eg. for previous example
* Reason: I don't know the population of Japan?
* Act: Use the search tool
* Observation: 125M
* Reason: I need to calculate the root of 125M
* Act: use the calculator
* Observation: result is 11180
* Response

#### Agent Execution Loop

This is the core cycle of how agents work:

1. Thought → The agent reasons internally. ("I need Japan’s population.")
2. Action → Calls a tool. ("Search for Japan population.")
3. Observation → Gets result from tool. ("125M.")
4. Thought → Decides next step. ("Now calculate the square root.")
5. Action → Uses calculator.
6. Observation → Gets the number.
7. Response → Returns final answer to user.

👉 This loop continues until the agent has enough info to answer.

## 4.5 Agents in Action

#### Zero-Shot ReAct Description Agent

* Zero-Shot → means the agent hasn’t been given examples ahead of time.
* ReAct → means it uses the Reason + Act loop we discussed.
* Description → it decides which tool to use based only on the natural language description of tools.

👉 Example tools given to the agent:

Calculator: "Useful for math operations like addition, subtraction, multiplication, division, or square roots."

Search: "Useful for finding real-world facts from the internet."

* Lets build a small agent that takes in names of some countries
* Finds their population
* Square roots them and store it in csv file

In [ ]:
from langchain.tools import tool
import math
import csv
# Calculator Tool
@tool
def square_root_tool(population: str) -> str:
    """
    This tool takes in a population as string and returns its square root in string format
    """
    value = int(population)
    return str(math.sqrt(value))

@tool
def write_to_csv(row: str) -> None:
    """
    This tool takes a row as string and writes it into a csv file.
    The input must be of the form city,population,square_root_of_population.
    eg: "Tokyo 125000000 11180"
    """
    item = row.split(" ")
    file = open("output.csv" , "a+")
    writer = csv.writer(file)
    writer.writerow(item)
    file.close()

@tool
def read_from_csv(dummy: str = "read") -> str:
    """
    This tool reads rows from the csv file and returns them as a string.
    Input is ignored (just pass anything).
    """
    with open("output.csv", "r") as file:
        data = file.read()
    return data

creating an agent that will take a couple of cities, find its population, find it square root and save it into csv file

In [ ]:
agent4 = initialize_agent(
    tools = [search, square_root_tool , write_to_csv, read_from_csv],
    llm = llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True
)

cities = "London, Tokyo, New York"
agent4.run(f"You are an agent that will fetch population count of these cities {cities}. After finding the population, find the square root of each city and write it into the csv file in the format city,population,root. Display the final csv file.")

In [ ]:
from langchain.agents import initialize_agent, AgentType


agent5 = initialize_agent(
    tools = [search],
    llm=llm2,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True,
    handle_parsing_error = True
)

agent5.invoke({"input":"Who is the current president of the United States"})

performing with LM studio model

In [ ]:
from langchain_openai import ChatOpenAI

llm3 = ChatOpenAI(
    model="google/gemma-3-4b",
    openai_api_key = 'lm-studio', # type: ignore
    openai_api_base="http://localhost:1234/v1" # type: ignore
)


In [ ]:
print(llm3.invoke("Who is the current president of USA"))

In [ ]:
from langchain.agents import initialize_agent, AgentType

react_prompt = """You are a helpful assistant that uses tools.
When answering, you must strictly follow this format:

Question: the input question to answer
Thought: your reasoning about what to do
Action: the exact name of the tool to use, nothing else
Action Input: the input to the tool, in plain text

If you already have the answer, respond like this:

Thought: I know the answer
Final Answer: <the answer>

Never add explanations after Action or Action Input.
"""

agent6 = initialize_agent(
    tools = [search],
    llm=llm3,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True,
    handle_parsing_error = True, 
)

agent6.invoke({"input":"Who is the current president of the United States"})

## 4.6 Agent Types


---

# 🧠 LangChain Agent Types

LangChain provides different **agent types**, each with its own reasoning style, memory handling, and tool support. Agents differ in **how they decide when and how to use tools**.

---

## 1. Zero-Shot ReAct Description

**Description**

* Uses the **ReAct (Reason + Act)** framework.
* LLM decides which tool to call based only on tool descriptions.
* No prior examples (“zero-shot”).

**When to Use**

* General-purpose agents.
* Tasks where the model needs to reason step by step and use tools.

**Limitations**

* Supports only **single-input tools** (`str -> str`).
* No memory between calls.

**Example**

```python
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)
```

---

## 2. Conversational ReAct Description

**Description**

* Same as Zero-Shot ReAct, but includes **memory** of the conversation.
* Lets you ask follow-up questions without repeating context.

**When to Use**

* Chatbots and assistants that need dialogue memory.

**Limitations**

* Still only works with **single-input tools**.

---

## 3. Self-Ask with Search

**Description**

* Agent decomposes a query into **sub-questions**.
* Uses **search tools** when it cannot answer from reasoning alone.

**When to Use**

* Knowledge Q\&A.
* Tasks where an LLM must fetch external information.

**Limitations**

* Not general-purpose — designed specifically for search-based reasoning.

---

## 4. Plan-and-Execute

**Description**

* Works in **two stages**:

  1. LLM **plans** the full set of steps.
  2. Then **executes** each step with sub-agents.

**When to Use**

* Complex, multi-step workflows.
* Scenarios requiring structured execution instead of ad-hoc reasoning.

**Limitations**

* Can be slower (planning + execution).
* Sometimes “over-plans.”

---

## 5. Structured Chat Agent

**Description**

* Unlike Zero-Shot, it supports **multi-input tools**.
* LLM calls tools with structured arguments (like a JSON dict).

**When to Use**

* When tools need multiple parameters (e.g., `{city: "Tokyo", population: 125000000}`).

**Limitations**

* Requires stricter prompts and well-defined tool schemas.

---

## 6. Functions Agent (OpenAI/Gemini)

**Description**

* Uses **function-calling APIs** of LLMs (OpenAI, Gemini, etc.).
* LLM outputs valid JSON with function arguments.
* Very reliable for tool use.

**When to Use**

* If your LLM supports function calling (e.g., GPT-4, Gemini).
* Production-grade tool invocation.

**Limitations**

* Dependent on LLM providers that implement function calling.

---

# 📊 Summary Table

| Agent Type                      | Multi-Input Tools? | Memory? | Best For                  |
| ------------------------------- | ------------------ | ------- | ------------------------- |
| Zero-Shot ReAct Description     | ❌ No               | ❌ No    | General tool use          |
| Conversational ReAct            | ❌ No               | ✅ Yes   | Chat assistants           |
| Self-Ask with Search            | ❌ No               | ❌ No    | Knowledge Q\&A            |
| Plan-and-Execute                | ❌ No               | ❌ No    | Complex workflows         |
| Structured Chat Agent           | ✅ Yes              | ❌ No    | Multi-input tools         |
| Functions Agent (OpenAI/Gemini) | ✅ Yes              | ❌ No    | Reliable structured calls |

---

⚡ **Tip**: Start with **Zero-Shot ReAct** (simplest), then move to **Structured Chat** or **Functions Agent** once your tools get more complex.

